In [2]:
import os

from groq import Groq

client = Groq(
    # This is the default and can be omitted
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are artificial intelligence (AI) systems designed to process and understand human language at incredibly high speeds. The importance of fast language models can be seen in several areas:

1. **Improved User Experience**: Fast language models enable applications to respond quickly to user input, making interactions feel more natural and intuitive. This is particularly important in real-time applications, such as chatbots, virtual assistants, and language translation software.
2. **Increased Efficiency**: By processing language quickly, businesses and organizations can automate tasks, such as data entry, text summarization, and sentiment analysis, freeing up human resources for more strategic and creative work.
3. **Enhanced Customer Service**: Fast language models can help power customer service chatbots, enabling them to respond quickly and accurately to customer inquiries, reducing wait times and improving customer satisfaction.
4. **Real-time Language Translation

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
apikey="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"
models=['llama-3.3-70b-versatile', 'mixtral-8x7b-32768']
parser_dateTime = DatetimeOutputParser()
chat = ChatGroq(temperature=0, groq_api_key=apikey, model_name=models[0])
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])



In [12]:
#date time parser
prompt_dateTime = PromptTemplate.from_template(
    template = "Answer the question.\n{format_instructions}\n{question}",
    input_vairables = ["question"],
    partial_variables = {"format_instructions": parser_dateTime.get_format_instructions()},
)
chain = prompt_dateTime | chat 
response=chain.invoke({"question": "When India got independence?"})
parser_dateTime.parse(response.content)


datetime.datetime(1947, 8, 15, 0, 0)

In [11]:
from langchain.output_parsers import CommaSeparatedListOutputParser
parser_list = CommaSeparatedListOutputParser()
prompt_list = PromptTemplate.from_template(
    template = "Answer the question.\n{format_instructions}\n{question}",
    input_vairables = ["question"],
    partial_variables = {"format_instructions": parser_list.get_format_instructions()},
)
chain = prompt_list | chat 

response=chain.invoke({"question": "provide Current Indian Cricket Team Players"})

returned_object = parser_list.parse(response.content)
returned_object


['Rohit Sharma',
 'Virat Kohli',
 'KL Rahul',
 'Shreyas Iyer',
 'Suryakumar Yadav',
 'Ishan Kishan',
 'Hardik Pandya',
 'Ravindra Jadeja',
 'Axar Patel',
 'Kuldeep Yadav',
 'Yuzvendra Chahal',
 'Jasprit Bumrah',
 'Mohammed Shami',
 'Mohammed Siraj',
 'Umran Malik',
 'Arshdeep Singh']

In [22]:
# Pydantic Output Parser
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class Author(BaseModel):
    name: str = Field(description="The name of the author")
    number: int = Field(description="The number of books written by the author")
    books: list[str] = Field(description="The list of books they wrote")

output_parser = PydanticOutputParser(pydantic_object=Author)
prompt_list = PromptTemplate.from_template(
    template = "Answer the question.\n{format_instructions}\n{question}",
    input_vairables = ["question"],
    partial_variables = {"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_list | chat 

response=chain.invoke({"question": "Tell me about books written by Peter Thiel"})
returned_object = output_parser.parse(response.content)
returned_object

Author(name='Peter Thiel', number=4, books=['The Diversity Myth', 'Zero to One'])

In [25]:
# In Langchain, runnables are a powerful abstraction representing any callable unit of work. 
# They're used to encapsulate and manage different kinds of tasks, including LLM calls, database queries, or calls to external APIs. 
# This allows us to chain together diverse operations in a consistent and manageable way, making it easy to construct complex workflows within Langchain applications.
from langchain_core.output_parsers import StrOutputParser

sentiment_template = PromptTemplate(
    input_variables=["feedback"],
    template="Determine the sentiment of this feedback and reply in one word as either 'Positive', 'Neutral', or 'Negative':\n\n{feedback}"
)

user_feedback = "The delivery was late, and the product was damaged when it arrived. However, the customer support team was very helpful in resolving the issue quickly."

chain = sentiment_template | chat | StrOutputParser()
feeback_sentiment = chain.invoke({"feedback": user_feedback})

print(feeback_sentiment)

Neutral.


In [ ]:
# Runnable Lambda

from langchain.schema.runnable import RunnableLambda

parse_template = PromptTemplate(
    input_variables=["raw_feedback"],
    template="Parse and clean the following customer feedback for key information:\n\n{raw_feedback}"
)

summary_template = PromptTemplate(
    input_variables=["parsed_feedback"],
    template="Summarize this customer feedback in one concise sentence:\n\n{parsed_feedback}"
)

sentiment_template = PromptTemplate(
    input_variables=["feedback"],
    template="Determine the sentiment of this feedback and reply in one word as either 'Positive', 'Neutral', or 'Negative':\n\n{feedback}"
)

format_parsed_output = RunnableLambda(lambda output: {"parsed_feedback": output})
format_summary_output = RunnableLambda(lambda output: {"feedback": output})

user_feedback = "The delivery was late, and the product was damaged when it arrived. and, the customer support team was very bad in resolving the issue."

chain = parse_template | chat | format_parsed_output | summary_template | chat | format_summary_output | sentiment_template| chat | StrOutputParser()
feedback_sentiment = chain.invoke({"raw_feedback": user_feedback})

print(feedback_sentiment)

Negative


In [43]:
#Conditional routing with a custom function#


parse_template = PromptTemplate(
    input_variables=["raw_feedback"],
    template="Parse and clean the following customer feedback for key information:\n\n{raw_feedback}"
)

summary_template = PromptTemplate(
    input_variables=["parsed_feedback"],
    template="Summarize this customer feedback in one concise sentence:\n\n{parsed_feedback}"
)

thankyou_template = PromptTemplate(
    input_variables=["feedback"],
    template="Given the feedback, draft a thank you message for the user and request them to leave a positive rating on our webpage:\n\n{feedback}"
)

details_template = PromptTemplate(
    input_variables=["feedback"],
    template="Given the feedback, draft a message for the user and request them provide more details about their concern:\n\n{feedback}"
)

apology_template = PromptTemplate(
    input_variables=["feedback"],
    template="Given the feedback, draft an apology message for the user and mention that their concern has been forwarded to the relevant department:\n\n{feedback}"
)

sentiment_template = PromptTemplate(
    input_variables=["feedback"],
    template="Determine the sentiment of this feedback and reply in one word as either 'Positive', 'Neutral', or 'Negative':\n\n{feedback}"
)

thankyou_chain = thankyou_template | chat | StrOutputParser()
details_chain = details_template | chat | StrOutputParser()
apology_chain = apology_template | chat | StrOutputParser()

def route(info):
    if "postive" in info['sentiment'].lower():
        return thankyou_chain
    elif "negative" in info['sentiment'].lower():
        return apology_chain
    else:
        return details_chain

#Neutral   
# user_feedback = "The delivery was late, and the product was damaged when it arrived. However, the customer support team was very helpful in resolving the issue quickly."
# Postive
# user_feedback = "The customer service was fantastic. The representative was friendly, knowledgeable, and resolved my issue quickly."

# Negative
user_feedback = "I was extremely disappointed with the customer service. The representative was unhelpful and rude."
format_parsed_output = RunnableLambda(lambda output: {"parsed_feedback": output})
summary_chain = parse_template | chat | format_parsed_output | summary_template | chat | StrOutputParser()
sentiment_chain = sentiment_template| chat | StrOutputParser()
summary = summary_chain.invoke({'raw_feedback' : user_feedback})
sentiment = sentiment_chain.invoke({'feedback': summary})

print("The summary of the user's message is:", summary)
print("The sentiment was classifed as:", sentiment)

full_chain = {"feedback": lambda x: x['feedback'], 'sentiment' : lambda x : x['sentiment']} | RunnableLambda(route)
print(full_chain.invoke({'feedback': summary, 'sentiment': sentiment}))


The summary of the user's message is: The customer had a negative experience with the customer service team, specifically with a representative they found to be unhelpful and rude, leading to strong feelings of disappointment.
The sentiment was classifed as: Negative.
Here's a draft apology message:

"Dear [User],

We are deeply sorry to hear that your recent interaction with our customer service team did not meet your expectations. We apologize for the unhelpful and rude behavior you experienced with one of our representatives, and we understand that this has caused you significant disappointment.

Please know that we take your concerns seriously and are committed to providing a better experience for our customers. We have forwarded your feedback to our Customer Service department, and they will be reviewing the incident to ensure that our representatives are providing the level of service and respect that you deserve.

Once again, we apologize for the poor service you received, and w

In [47]:
# Tools in Langchain
from langchain_core.tools import tool
@tool
def calculate_discount(price: float, discount_percentage: float) -> float:
    """
    Calculates the final price after applying a discount.

    Args:
        price (float): The original price of the item.
        discount_percentage (float): The discount percentage (e.g., 20 for 20%).

    Returns:
        float: The final price after the discount is applied.
    """
    if not (0 <= discount_percentage <= 100):
        raise ValueError("Discount percentage must be between 0 and 100")

    discount_amount = price * (discount_percentage / 100)
    final_price = price - discount_amount
    return final_price

print(calculate_discount.name)
print(calculate_discount.description)
print(calculate_discount.args)
print(calculate_discount.invoke({"price":100, "discount_percentage": 15}))

calculate_discount
Calculates the final price after applying a discount.

Args:
    price (float): The original price of the item.
    discount_percentage (float): The discount percentage (e.g., 20 for 20%).

Returns:
    float: The final price after the discount is applied.
{'price': {'title': 'Price', 'type': 'number'}, 'discount_percentage': {'title': 'Discount Percentage', 'type': 'number'}}
85.0


In [59]:
#Tool Binding
llm_with_tools = chat.bind_tools([calculate_discount])
result=llm_with_tools.invoke("I have a T Shirt with price Rs 1000 and it have around 75% of discount what is the final price?")
arguments=result.tool_calls[0]['args']
print(calculate_discount.invoke(arguments))

250.0
